# Input paths

In [ ]:
import os

PRETRAINED_ENCODER_PATH = '../input/airbus-ship-detection-resnet34-256px'
PRETRAINED_ENCODER_NAME = 'ResNet34_ship_detection_256_4.h5'

# PRETRAINED_SEG_MODEL_PATH = '../input/airbus-seg-model-256-sr-1/seg_model_256_ships_ratio_1.hdf5'
PRETRAINED_SEG_MODEL_PATH = '../input/airbus-seg-model-256-sr-0741/seg_model_768_ships_ratio_0741_5hr.hdf5'

SHIP_DIR = '../input/airbus-ship-detection'
TRAIN_IMAGE_DIR = os.path.join(SHIP_DIR, 'train_v2')
TEST_IMAGE_DIR = os.path.join(SHIP_DIR, 'test_v2')

MASKS_DF = 'train_ship_segmentations_v2.csv'

In [ ]:
masks = pd.read_csv(os.path.join(SHIP_DIR, MASKS_DF))
print(masks.shape[0], 'masks found')
print(masks['ImageId'].value_counts().shape[0])
masks.head()

## Model Parameters
We might want to adjust these later (or do some hyperparameter optimizations)

In [ ]:
BATCH_SIZE = 4
EDGE_CROP = 16
NB_EPOCHS = 5
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# number of validation images to use
VALID_IMG_COUNT = 400
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 200
AUGMENT_BRIGHTNESS = False

IMG_SIZE = 768

In [ ]:
!ls ../input

In [ ]:
# import imgaug
# imgaug.__version__

In [ ]:
!pip install --upgrade imgaug

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from skimage.io import imread
from cv2 import imread 
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
import time

from keras.utils import Sequence
from sklearn.metrics import fbeta_score
from imgaug import augmenters as iaa

import cv2

from skimage.util import montage
# from skimage.util.montage2d import montage2d as montage was this for older skimage

montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)

import gc; gc.enable() # memory is tight

from skimage.morphology import label


In [ ]:

def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)

def masks_as_image_to_dict(row):
    in_mask_list = [row['EncodedPixels']]
    image_name = row['ImageId']
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
            
    mask = np.expand_dims(all_masks, -1)
    try:
        dict_masks[image_name].append(mask)
    except:
        dict_masks[image_name] = [mask]
#     return np.expand_dims(all_masks, -1)



# Split into training and validation groups
We stratify by the number of boats appearing so we have nice balances in each set

In [ ]:
masks['ships'] = masks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
unique_img_ids = masks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()
unique_img_ids['has_ship'] = unique_img_ids['ships'].map(lambda x: 1.0 if x>0 else 0.0)
unique_img_ids['has_ship_vec'] = unique_img_ids['has_ship'].map(lambda x: [x])
# some files are too small/corrupt
unique_img_ids['file_size_kb'] = unique_img_ids['ImageId'].map(lambda c_img_id: 
                                                               os.stat(os.path.join(TRAIN_IMAGE_DIR, 
                                                                                    c_img_id)).st_size/1024)
unique_img_ids = unique_img_ids[unique_img_ids['file_size_kb']>50] # keep only 50kb files
unique_img_ids['file_size_kb'].hist()
masks.drop(['ships'], axis=1, inplace=True)
unique_img_ids.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split
train_ids, valid_ids = train_test_split(unique_img_ids, 
                 test_size = 0.05, 
                 stratify = unique_img_ids['ships'])
train_df = pd.merge(masks, train_ids)
valid_df = pd.merge(masks, valid_ids)
print(train_df.shape[0], 'training masks')
print(valid_df.shape[0], 'validation masks')

# Balance whole train_df dataset

In [ ]:
train_df.head()

In [ ]:
# seg_df_copy = masks.copy()    
seg_df = train_df


ships_ratio_all = 0.6  ## WRONG coef  (0.5 min -> more like non_ship/ship ratio)  0.65=0.76369    ////// HERE LABELS 0.6=0.84685!!0.6=0.741 ship/no_ship images
ship_images_cnt = seg_df[seg_df['has_ship'] == 1].shape[0]
x_coef = seg_df.shape[0]/ship_images_cnt - (1/ships_ratio_all) # just for formula

exclude_non_ship_images_cnt = seg_df.shape[0]-int(x_coef*ship_images_cnt)
list_ids_exclude = seg_df[seg_df['has_ship']!= 1].sample(exclude_non_ship_images_cnt)['ImageId']
seg_df = seg_df[~seg_df['ImageId'].isin(list_ids_exclude)]
print("ship labels (few for multiple ships per image):", seg_df[seg_df['has_ship'] == 1]['ImageId'].nunique())
print("non ship labels:", seg_df[seg_df['has_ship'] != 1].shape[0])
print("Proportion has_ship=1:", seg_df[seg_df['has_ship'] == 1]['ImageId'].nunique()/seg_df['ImageId'].nunique())
train_df = seg_df
train_df.shape

In [ ]:
# # train_df = train_df.sample(40*1000)
# train_df = train_df[:20000]
# print("ship labels (few for multiple ships per image):", train_df[train_df['has_ship'] == 1].shape[0])
# print("non ship labels:", train_df[train_df['has_ship'] != 1].shape[0])
# train_df[train_df['has_ship'] == 1]['ImageId'].nunique()/train_df['ImageId'].nunique()


In [ ]:
# train_df['grouped_ship_count'] = train_df['ships'].map(lambda x: (x+1)//2).clip(0, 7)
# def sample_ships(in_df, base_rep_val=1500):
#     if in_df['ships'].values[0]==0:
#         return in_df.sample(base_rep_val//3) # even more strongly undersample no ships
#     else:
#         return in_df.sample(base_rep_val, replace=(in_df.shape[0]<base_rep_val))
    
# balanced_train_df = train_df.groupby('grouped_ship_count').apply(sample_ships)
# balanced_train_df['ships'].hist(bins=np.arange(10))

In [ ]:
# balanced_train_df.shape

In [ ]:
# import keras



class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, in_df, is_validation=False, ships_ratio=None, batch_size=32, crop_size=256, shuffle=True):
        'Initialization'
        self.in_df = in_df
        self.is_validation = is_validation
        self.ships_ratio = ships_ratio 
        self.batch_size = batch_size
        self.crop_size = crop_size
        self.shuffle = shuffle
        
        if ships_ratio == 1:
            #only has_ship images
            self.in_df = in_df[in_df['has_ship']==1.0]
        if ships_ratio is None:
            #then assign default ship/no_ship distribution
            self.ships_ratio = in_df[in_df['has_ship']==1.0]['ImageId'].nunique()/in_df['ImageId'].nunique()
        self.list_IDs = self.in_df['ImageId'].unique()

        print("is_validation: {}, ships_ratio:{}".format(is_validation, round(self.ships_ratio, 3)))
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def augmentor(self, X, y):
        """
        dg_args = dict(featurewise_center = False, 
                  samplewise_center = False,
                  rotation_range = 15, ---------
                  width_shift_range = 0.1, 
                  height_shift_range = 0.1, 
                  shear_range = 0.01,
                  zoom_range = [0.9, 1.25],  
                  horizontal_flip = True, 
                  vertical_flip = True,
                  fill_mode = 'reflect',
                   data_format = 'channels_last')
        """      
        geometric_augs = [
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.3), # vertically flip 30% of all images
            iaa.Affine(rotate=(-15,15)) # random rotate on +-15 degree
        ]
        visual_augs = [
            iaa.Add((-5, 5), per_channel=0.5), # change brightness of images
            iaa.AdditiveGaussianNoise(scale=0.02*255),
            iaa.GaussianBlur(sigma=[0, 0, 0, 0.01, 0.05, 0.2, 0.5, 0.8, 1, 1.5])
        ]
        seq_aug = iaa.Sequential(geometric_augs+visual_augs)
        X, y = seq_aug(images=X, segmentation_maps=y)
        return X, y
            
    def _get_smart_cropped_image_mask(self, c_img, c_mask, c_masks):
        """
        Function takes:
        @c_img: input fullsize rgb image (3 dims)
        @c_mask: input fullsize mask (1 dim)
        @c_masks: dataframe with ships' masks form this image
        (takes 3-4ms for image with ships) an
        d ~0.1ms for empty image
        """
        crop_margin = int(self.crop_size/2)
#         try:
        if c_masks['has_ship'].values[0]==1:
            #pick random ship from image
            c_mask_ship = masks_as_image(np.array([np.random.choice(c_masks['EncodedPixels'].values)]))
            mask_ship_bw = c_mask_ship[:,:,0]
            #find indexes where mask is 1
            arr_indexes = np.where(mask_ship_bw == 1)
            center_x = int((arr_indexes[1].min()+arr_indexes[1].max())/2)
            center_y = int((arr_indexes[0].min()+arr_indexes[0].max())/2)
        else:
            center_x = np.random.uniform(0, IMG_SIZE)
            center_y = np.random.uniform(0, IMG_SIZE)
        # we got ~ center of picked ship

        #make ship position uniformly distributed in image crop, not only at the center
        #minimum half of ship is present in image
        center_x = int(center_x+(crop_margin*np.random.uniform(low=-0.5, high=0.5)))
        center_y = int(center_y+(crop_margin*np.random.uniform(low=-0.5, high=0.5)))

        #get center of crop understanding border of the image
        center_crop_x = center_x
        center_crop_y = center_y
        if center_x-crop_margin<0:
            center_crop_x = crop_margin
        if center_x+crop_margin>IMG_SIZE:
            center_crop_x = IMG_SIZE-crop_margin
        if center_y-crop_margin<0:
            center_crop_y = crop_margin
        if center_y+crop_margin>IMG_SIZE:
            center_crop_y = IMG_SIZE-crop_margin

        #final crope coordinates
        x1 = center_crop_x-crop_margin
        x2 = center_crop_x+crop_margin
        y1 = center_crop_y-crop_margin
        y2 = center_crop_y+crop_margin

        c_img  =  c_img[y1:y2,x1:x2,:]
        c_mask = c_mask[y1:y2,x1:x2,:]
        return c_img, c_mask


    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.crop_size, self.crop_size, 3), dtype='float32')
        y = np.empty((self.batch_size, self.crop_size, self.crop_size, 1), dtype='uint8')
        
        # Generate data
        for i, c_img_id in enumerate(list_IDs_temp):
            c_masks = self.in_df[self.in_df['ImageId']==c_img_id]
            
            rgb_path = os.path.join(TRAIN_IMAGE_DIR, c_img_id)
            c_img = imread(rgb_path)  # 13ms
            
            c_mask = masks_as_image(c_masks['EncodedPixels'].values)
            
            #smart image crop
            if self.crop_size != 768:
                c_img, c_mask = self._get_smart_cropped_image_mask(c_img, c_mask, c_masks) # 1-4ms
            
            # Store sample
            X[i,] = c_img
            # Store class
            y[i] = c_mask
#             print("time for item: ms", 1000*(time.time()-time_start_item))  # 35ms

        return X, y
    
    def __getitem__(self, index):
        time_start_batch = time.time()
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)  # ~1.5s

        # preprocess and augment data
#         time_start_aug = time.time()
        if not self.is_validation:
            X, y = self.augmentor(X, y)  # 0.001 ms
#         print("\n\ntime for aug: ms", 1000*(time.time()-time_start_aug))

#         print("\n\n\ntime for batch: ms", 1000*(time.time()-time_start_batch))
        return X/255.0, y


In [ ]:
train_df.head()

In [ ]:
# training_generator = DataGenerator(train_df, is_validation=False, ships_ratio=1, batch_size=32, crop_size=256, shuffle=True)
# X, y = next(iter(training_generator))

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

# dg_args = dict(featurewise_center = False, 
#                   samplewise_center = False,
#                   rotation_range = 15, 
#                   width_shift_range = 0.1, 
#                   height_shift_range = 0.1, 
#                   shear_range = 0.01,
#                   zoom_range = [0.9, 1.25],  
#                   horizontal_flip = True, 
#                   vertical_flip = True,
#                   fill_mode = 'reflect',
#                    data_format = 'channels_last')
# # brightness can be problematic since it seems to change the labels differently from the images 
# if AUGMENT_BRIGHTNESS:
#     dg_args[' brightness_range'] = [0.5, 1.5]
# image_gen = ImageDataGenerator(**dg_args)

# if AUGMENT_BRIGHTNESS:
#     dg_args.pop('brightness_range')
# label_gen = ImageDataGenerator(**dg_args)

# def create_aug_generator(in_gen, seed = 42):
#     """
#     Wrapper around previous generator that streams cropped batched images
#     and make data augmentation on images
#     """
#     np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
#     for in_x, in_y in in_gen:
#         seed = np.random.choice(range(9999))
#         # keep the seeds syncronized otherwise the augmentation to the images is different from the masks
#         g_x = image_gen.flow(255*in_x, 
#                              batch_size = in_x.shape[0], 
#                              seed = seed, 
#                              shuffle=True)
#         g_y = label_gen.flow(in_y, 
#                              batch_size = in_x.shape[0], 
#                              seed = seed, 
#                              shuffle=True)

#         yield next(g_x)/255.0, next(g_y)

In [ ]:
def change_model_input_shape(model, loss, lr, new_input_shape=(None, 768, 768, 3)):
    # replace input shape of first layer
    model._layers[0].batch_input_shape = new_input_shape

    # rebuild model architecture by exporting and importing via json
    new_model = keras.models.model_from_json(model.to_json())

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
        except:
            print("Could not transfer weights for layer {}".format(layer.name))
    
    # train
    del model
#     adam = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)
    optimizer = Adam(lr, decay=1e-6)
#     optimizer = SGD(learning_rate=lr)
    new_model.compile(optimizer=optimizer, loss=loss, metrics=[dice_coef, 'binary_accuracy', true_positive_rate])
    return new_model
    
def change_input_shape(model, loss, lr=0.0001, new_input_size=256, batch_size=64, ships_ratio=None):
    print("Changing model's input shape to:", new_input_size)
    model = change_model_input_shape(model, loss, lr=lr, new_input_shape=(None, new_input_size, new_input_size, 3))
    print("Creating datagenerators...")

#     train_generator = create_aug_generator(make_image_gen(train_df, batch_size, new_input_size, ships_ratio))
    train_generator = DataGenerator(train_df, is_validation=False, ships_ratio=ships_ratio, batch_size=batch_size, crop_size=new_input_size, shuffle=True)
    validation_generator = DataGenerator(valid_df, is_validation=True, ships_ratio=ships_ratio, batch_size=batch_size, crop_size=new_input_size)
#     validation_generator = next(make_image_gen(valid_df, valid_df['ImageId'].nunique(), crop_size=new_input_size, ships_ratio=None))

    
    return model, train_generator, validation_generator

In [ ]:
gc.collect()

# Build ResNet34+ Unet Model
Here we use a slight deviation on the U-Net standard

In [ ]:
import numpy as np
import pandas as pd
import six

from random import randint

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

from sklearn.model_selection import train_test_split

from skimage.transform import resize


from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam, SGD
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout,BatchNormalization
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from tqdm import tqdm_notebook
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras.engine import InputSpec
from keras import backend as K
from keras.regularizers import l2

from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate,add
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D

## Create Upsample layer


In [ ]:
from keras.layers import Conv2DTranspose
from keras.layers import UpSampling2D
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Concatenate

def handle_block_names_decode(stage):
    conv_name = 'decoder_stage{}_conv'.format(stage)
    bn_name = 'decoder_stage{}_bn'.format(stage)
    relu_name = 'decoder_stage{}_relu'.format(stage)
    up_name = 'decoder_stage{}_upsample'.format(stage)
    return conv_name, bn_name, relu_name, up_name


def Upsample2D_block(filters, stage, kernel_size=(3,3), upsample_rate=(2,2),
                     batchnorm=False, skip=None):

    def layer(input_tensor):

        conv_name, bn_name, relu_name, up_name = handle_block_names_decode(stage)

        x = UpSampling2D(size=upsample_rate, name=up_name)(input_tensor)

        if skip is not None:
            x = Concatenate()([x, skip])

        x = Conv2D(filters, kernel_size, padding='same', name=conv_name+'1')(x)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'1')(x)
        x = Activation('relu', name=relu_name+'1')(x)

        x = Conv2D(filters, kernel_size, padding='same', name=conv_name+'2')(x)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'2')(x)
        x = Activation('relu', name=relu_name+'2')(x)

        return x
    return layer


def Transpose2D_block(filters, stage, kernel_size=(3,3), upsample_rate=(2,2),
                      transpose_kernel_size=(4,4), batchnorm=False, skip=None):

    def layer(input_tensor):

        conv_name, bn_name, relu_name, up_name = handle_block_names_decode(stage)

        x = Conv2DTranspose(filters, transpose_kernel_size, strides=upsample_rate,
                            padding='same', name=up_name)(input_tensor)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'1')(x)
        x = Activation('relu', name=relu_name+'1')(x)

        if skip is not None:
            x = Concatenate()([x, skip])

        x = Conv2D(filters, kernel_size, padding='same', name=conv_name+'2')(x)
        if batchnorm:
            x = BatchNormalization(name=bn_name+'2')(x)
        x = Activation('relu', name=relu_name+'2')(x)

        return x
    return layer

## Define Unet model (Decoder)

In [ ]:
def build_unet(backbone, classes, last_block_filters, skip_layers,
               n_upsample_blocks=5, upsample_rates=(2,2,2,2,2),
               block_type='upsampling', activation='sigmoid',
               **kwargs):

    backbone_input = backbone.input
    x = backbone.output
#     print("backbone.output:", x)
    if block_type == 'transpose':
        up_block = Transpose2D_block
    else:
        up_block = Upsample2D_block

    # convert layer names to indices
    skip_layers = ([get_layer_number(backbone, l) if isinstance(l, str) else l
                    for l in skip_layers])
    for i in range(n_upsample_blocks):

        # check if there is a skip connection
        if i < len(skip_layers):
            print(backbone.layers[skip_layers[i]])
            print(backbone.layers[skip_layers[i]].output)
            skip = backbone.layers[skip_layers[i]].output
        else:
            skip = None

        up_size = (upsample_rates[i], upsample_rates[i])
        filters = last_block_filters * 2**(n_upsample_blocks-(i+1))

        x = up_block(filters, i, upsample_rate=up_size, skip=skip, **kwargs)(x)

    print("created upsample blocks")
    if classes < 2:
        activation = 'sigmoid'

    x = Conv2D(classes, (3,3), padding='same', name='final_conv')(x)
    x = Activation(activation, name=activation)(x)

    print("creating Model instance")
    model = Model(backbone_input, x)

    return model

## Built ResNet34 model
* Reference this [github](https://github.com/qubvel/classification_models/blob/master/classification_models/resnet/)

In [ ]:
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Add
from keras.layers import ZeroPadding2D

def handle_block_names(stage, block):
    name_base = 'stage{}_unit{}_'.format(stage + 1, block + 1)
    conv_name = name_base + 'conv'
    bn_name = name_base + 'bn'
    relu_name = name_base + 'relu'
    sc_name = name_base + 'sc'
    return conv_name, bn_name, relu_name, sc_name


def basic_identity_block(filters, stage, block):

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer


def basic_conv_block(filters, stage, block, strides=(2, 2)):

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        shortcut = Conv2D(filters, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def conv_block(filters, stage, block, strides=(2, 2)):
    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        shortcut = x
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), strides=strides, name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        shortcut = Conv2D(filters*4, (1, 1), name=sc_name, strides=strides, **conv_params)(shortcut)
        x = Add()([x, shortcut])
        return x

    return layer


def identity_block(filters, stage, block):

    def layer(input_tensor):
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = Activation('relu', name=relu_name + '1')(x)
        x = Conv2D(filters, (1, 1), name=conv_name + '1', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = Activation('relu', name=relu_name + '2')(x)
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = Conv2D(filters, (3, 3), name=conv_name + '2', **conv_params)(x)

        x = BatchNormalization(name=bn_name + '3', **bn_params)(x)
        x = Activation('relu', name=relu_name + '3')(x)
        x = Conv2D(filters*4, (1, 1), name=conv_name + '3', **conv_params)(x)

        x = Add()([x, input_tensor])
        return x

    return layer

In [ ]:
def get_conv_params(**params):
    default_conv_params = {
        'kernel_initializer': 'glorot_uniform',
        'use_bias': False,
        'padding': 'valid',
    }
    default_conv_params.update(params)
    return default_conv_params


def get_bn_params(**params):
    default_bn_params = {
        'axis': 3,
        'momentum': 0.99,
        'epsilon': 2e-5,
        'center': True,
        'scale': True,
    }
    default_bn_params.update(params)
    return default_bn_params

In [ ]:
import keras.backend as K
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import GlobalAveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Dense
from keras.models import Model
from keras.engine import get_source_inputs

import keras
from distutils.version import StrictVersion

if StrictVersion(keras.__version__) < StrictVersion('2.2.0'):
    from keras.applications.imagenet_utils import _obtain_input_shape
else:
    from keras_applications.imagenet_utils import _obtain_input_shape
    
def build_resnet(
     repetitions=(2, 2, 2, 2),
     include_top=True,
     input_tensor=None,
     input_shape=None,
     classes=1000,
     block_type='usual'):

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=197,
                                      data_format='channels_last',
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape, name='data')
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    
    # get parameters for model layers
    no_scale_bn_params = get_bn_params(scale=False)
    bn_params = get_bn_params()
    conv_params = get_conv_params()
    init_filters = 64

    if block_type == 'basic':
        conv_block = basic_conv_block
        identity_block = basic_identity_block
    else:
        conv_block = usual_conv_block
        identity_block = usual_identity_block
    
    # resnet bottom
    x = BatchNormalization(name='bn_data', **no_scale_bn_params)(img_input)
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(init_filters, (7, 7), strides=(2, 2), name='conv0', **conv_params)(x)
    x = BatchNormalization(name='bn0', **bn_params)(x)
    x = Activation('relu', name='relu0')(x)
    x = ZeroPadding2D(padding=(1, 1))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', name='pooling0')(x)
    
    # resnet body
    for stage, rep in enumerate(repetitions):
        for block in range(rep):
            
            filters = init_filters * (2**stage)
            
            # first block of first stage without strides because we have maxpooling before
            if block == 0 and stage == 0:
                x = conv_block(filters, stage, block, strides=(1, 1))(x)
                
            elif block == 0:
                x = conv_block(filters, stage, block, strides=(2, 2))(x)
                
            else:
                x = identity_block(filters, stage, block)(x)
                
    x = BatchNormalization(name='bn1', **bn_params)(x)
    x = Activation('relu', name='relu1')(x)

    # resnet top
    if include_top:
        x = GlobalAveragePooling2D(name='pool1')(x)
        x = Dense(classes, name='fc1')(x)
        x = Activation('softmax', name='softmax')(x)

    # Ensure that the model takes into account any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
        
    # Create model.
    model = Model(inputs, x)

    return model

## Load pretrain Weight by using ImageNet trained
* Reference this [GITHUB](https://github.com/qubvel/classification_models/blob/9e438e2e133897b115148c737abdda3e1db31787/classification_models/weights.py)

In [ ]:
def load_model_weights(model):
    weights_path = os.path.join(PRETRAINED_ENCODER_PATH, PRETRAINED_ENCODER_NAME)

    from keras.models import load_model
    pretrained_resnet34 = load_model(weights_path)
    # transfer encoder weights without 3 last layers (dropout, pooling, head:1 sigmoid)
    for i, layer in enumerate(model.layers[:-3]):
        try:
            layer.set_weights(pretrained_resnet34.layers[i].get_weights())
        except:
            print("Could not transfer weights for layer {}".format(layer.name))


## Buil Unet model base on ResNet34

In [ ]:
#Freeze Encoder weight if needed

def freeze_model(model):
    for layer in model.layers:
        layer.trainable = False
    return


In [ ]:
def UResNet34(input_shape=(None, None, 3), classes=1, decoder_filters=16, decoder_block_type='upsampling',
                       encoder_weights=None, input_tensor=None, activation='sigmoid', **kwargs):

    backbone = build_resnet(input_tensor=None,
                         input_shape=input_shape,
                         repetitions=(3, 4, 6, 3),
                         classes=classes,
                         include_top=False,
                         block_type='basic')
    backbone.name = 'resnet34'
    
    if encoder_weights == True:
        load_model_weights(backbone)
        print("Loaded weight for the encoder!")
    
    skip_connections = list([129, 74, 37, 5]) # for resnet 34
    model = build_unet(backbone, classes, decoder_filters,
                       skip_connections, block_type=decoder_block_type,
                       activation=activation, **kwargs)
    model.name = 'u-resnet34'
    
#     print("Freezing backbone model's layers")
#     freeze_model(backbone)

    return model

## Define Loss function

In [ ]:
from keras.losses import binary_crossentropy
from keras import backend as K

"""
Here is a dice loss for keras which is smoothed to approximate a linear (L1) loss.
It ranges from 1 to 0 (no error), and returns results similar to binary crossentropy
"""
def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)
def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred = K.cast(y_pred, 'float32')
    y_pred_f = K.cast(K.greater(K.flatten(y_pred), 0.5), 'float32')
    intersection = y_true_f * y_pred_f
    score = 2. * K.sum(intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return score

def dice_loss(y_true, y_pred):
    return dice_coef_loss(y_true, y_pred)
#     smooth = 1.
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
#     intersection = y_true_f * y_pred_f
#     score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
#     return 1. - score

def bce_dice_loss(y_true, y_pred):
    return 2*binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def bce_logdice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - K.log(1. - dice_loss(y_true, y_pred))

def weighted_bce_loss(y_true, y_pred, weight):
    epsilon = 1e-7
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    logit_y_pred = K.log(y_pred / (1. - y_pred))
    loss = weight * (logit_y_pred * (1. - y_true) + 
                     K.log(1. + K.exp(-K.abs(logit_y_pred))) + K.maximum(-logit_y_pred, 0.))
    return K.sum(loss) / K.sum(weight)

def weighted_dice_loss(y_true, y_pred, weight):
    smooth = 1.
    w, m1, m2 = weight, y_true, y_pred
    intersection = (m1 * m2)
    score = (2. * K.sum(w * intersection) + smooth) / (K.sum(w * m1) + K.sum(w * m2) + smooth)
    loss = 1. - K.sum(score)
    return loss

def weighted_bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    # if we want to get same size of output, kernel size must be odd
    averaged_mask = K.pool2d(
            y_true, pool_size=(50, 50), strides=(1, 1), padding='same', pool_mode='avg')
    weight = K.ones_like(averaged_mask)
    w0 = K.sum(weight)
    weight = 5. * K.exp(-5. * K.abs(averaged_mask - 0.5))
    w1 = K.sum(weight)
    weight *= (w0 / w1)
    loss = weighted_bce_loss(y_true, y_pred, weight) + dice_loss(y_true, y_pred)
    return loss

In [ ]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
def dice_p_bce(in_gt, in_pred):
    return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)
def true_positive_rate(y_true, y_pred):
    return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/K.sum(y_true)
# def true_positive_rate(y_true, y_pred):
#     return K.sum(K.flatten(y_true)*K.flatten(K.round(y_pred)))/(K.sum(y_true) + K.epsilon())


# F2 score eval from iafoss submission notebook

In [ ]:
def IoU(pred, targs):
    pred = (pred > 0.5).astype(float)
    intersection = (pred*targs).sum()
    return intersection / ((pred+targs).sum() - intersection + 1.0)

def get_score(pred, true):
    n_th = 10
    b = 4
    thresholds = [0.5 + 0.05*i for i in range(n_th)]
    n_masks = len(true)
    n_pred = len(pred)
    ious = []
    score = 0
    for mask in true:
        buf = []
        for p in pred: buf.append(IoU(p,mask))
        ious.append(buf)
    for t in thresholds:   
        tp, fp, fn = 0, 0, 0
        for i in range(n_masks):
            match = False
            for j in range(n_pred):
                if ious[i][j] > t: match = True
            if not match: fn += 1
        
        for j in range(n_pred):
            match = False
            for i in range(n_masks):
                if ious[i][j] > t: match = True
            if match: tp += 1
            else: fp += 1
        score += ((b+1)*tp)/((b+1)*tp + b*fn + fp)       
    return score/n_th

def split_mask(mask):
    """
     return score/n_th
In this competition we should submit and individual mask for each identified ship. The simplest way to do it is splitting the total mask into individual ones based on the connectivity of detected objects.
"""
    threshold = 0.5
    threshold_obj = 30 #ignor predictions composed of "threshold_obj" pixels or less
    labled,n_objs = ndimage.label(mask > threshold)
    result = []
    for i in range(n_objs):
        obj = (labled == i + 1).astype(int)
        if(obj.sum() > threshold_obj): result.append(obj)
    return result

def get_mask_ind(img_id, df, shape = (768,768)): #return mask for each ship
    masks = df.loc[img_id]['EncodedPixels']
    if(type(masks) == float): return []
    if(type(masks) == str): masks = [masks]
    result = []
    for mask in masks:
        img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
        s = mask.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1
        result.append(img.reshape(shape).T)
    return result

class Score_eval():
    def __init__(self):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        self.score, self.count = 0.0, 0
        
    def put(self,pred,name):
        true = get_mask_ind(name, self.segmentation_df)
        self.score += get_score(pred,true)
        self.count += 1
        
    def evaluate(self):
        return self.score/self.count

## F2 eval score callback

In [ ]:
from keras.callbacks import Callback


# Notice that this callback only works with Keras 2.0.0
START = 0.5
END = 0.95
STEP = 0.05
N_STEPS = int((END - START) / STEP) + 2
DEFAULT_THRESHOLDS = np.linspace(START, END, N_STEPS)
DEFAULT_LOGS = {}
FBETA_METRIC_NAME = "val_f2"

from keras import backend as K


class FBetaMetricCallback(Callback):

    def __init__(self, validation_generator, validation_steps, beta=2, thresholds=DEFAULT_THRESHOLDS):
        self.validation_generator = validation_generator
        self.validation_steps = validation_steps
        self.beta = beta
        self.thresholds = thresholds
        # Will be initialized when the training starts
        self.val_fbeta = None

    def on_train_begin(self, logs=DEFAULT_LOGS):
        """ This is where the validation Fbeta
        validation scores will be saved during training: one value per
        epoch.
        """
        self.val_fbeta = []
            
    def _fbeta_score_OLD(self, y_true, y_pred, beta, threshold_shift=0):
        beta = 2

        # just in case of hipster activation at the final layer
        y_pred = K.clip(y_pred, 0, 1)

        # shifting the prediction threshold from .5 if needed
        y_pred_bin = K.round(y_pred + threshold_shift)

        tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
        fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
        fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)

        beta_squared = beta ** 2
        return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())


    def _score_per_threshold(self, predictions, targets, threshold):
        """ Compute the Fbeta score per threshold.
        """
        # Notice that here I am using the sklearn fbeta_score function.
        # You can read more about it here:
        # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.fbeta_score.html
        thresholded_predictions = (predictions > threshold).astype('int32')
        thresholded_predictions = thresholded_predictions.flatten()
        targets = targets.flatten()

        return fbeta_score(targets, thresholded_predictions, average=None, beta=self.beta)
#     return _fbeta_score(targets, thresholded_predictions, beta=self.beta)

    def on_epoch_end(self, epoch, logs=DEFAULT_LOGS):
        val_predictions = self.model.predict_generator(self.validation_generator, steps=self.validation_steps)#,steps = nb_samples)
        
        val_targets = []
        for i in range(self.validation_steps):
            val_targets.extend(next(iter(self.validation_generator))[1])
        val_targets = np.array(val_targets)
#         val_targets = self.validation_data[1]
        
        _val_fbeta = get_score(val_predictions, val_targets)
#         _val_fbeta = np.mean([self._score_per_threshold(val_predictions,
#                                                         val_targets, threshold)
#                               for threshold in self.thresholds])
        self.val_fbeta.append(_val_fbeta)
        print("Current F{} metric is: {}".format(str(self.beta), str(_val_fbeta)))
        return

    def on_train_end(self, logs=DEFAULT_LOGS):
        """ Assign the validation Fbeta computed metric to the History object.
        """
        self.model.history.history[FBETA_METRIC_NAME] = self.val_fbeta
    

# LRFinder

In [ ]:
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

class LRFinder(Callback):
    def __init__(self, min_lr, max_lr, mom=0.9, stop_multiplier=None, 
                 reload_weights=True, batches_lr_update=5):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.mom = mom
        self.reload_weights = reload_weights
        self.batches_lr_update = batches_lr_update
        if stop_multiplier is None:
            self.stop_multiplier = -20*self.mom/3 + 10 # 4 if mom=0.9
                                                       # 10 if mom=0
        else:
            self.stop_multiplier = stop_multiplier
        
    def on_train_begin(self, logs={}):
        p = self.params
        try:
            n_iterations = p['epochs']*p['samples']//p['batch_size']
        except:
            n_iterations = p['steps']*p['epochs']
            
        self.learning_rates = np.geomspace(self.min_lr, self.max_lr, \
                                           num=n_iterations//self.batches_lr_update+1)
        self.losses=[]
        self.iteration=0
        self.best_loss=0
        if self.reload_weights:
            self.model.save_weights('tmp.hdf5')
        
    
    def on_batch_end(self, batch, logs={}):
        loss = logs.get('loss')
        
        if self.iteration!=0: # Make loss smoother using momentum
            loss = self.losses[-1]*self.mom+loss*(1-self.mom)
        
        if self.iteration==0 or loss < self.best_loss: 
                self.best_loss = loss
                
        if self.iteration%self.batches_lr_update==0: # Evaluate each lr over 5 epochs
            
            if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
          
            lr = self.learning_rates[self.iteration//self.batches_lr_update]            
            K.set_value(self.model.optimizer.lr, lr)

            self.losses.append(loss)            

        if loss > self.best_loss*self.stop_multiplier: # Stop criteria
            self.model.stop_training = True
                
        self.iteration += 1
    
    def on_train_end(self, logs=None):
        if self.reload_weights:
                self.model.load_weights('tmp.hdf5')
                
        plt.figure(figsize=(12, 6))
        plt.plot(self.learning_rates[:len(self.losses)], self.losses)
        plt.xlabel("Learning Rate")
        plt.ylabel("Loss")
        plt.xscale('log')
        plt.show()
        

In [ ]:
gc.collect()

## Set Training Check Point

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('seg_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_dice_coef', verbose=1, 
                             save_best_only=True, mode='max', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, 
                                   patience=3, 
                                   verbose=1, mode='max', epsilon=0.0001, cooldown=2, min_lr=1e-6)
early = EarlyStopping(monitor="val_dice_coef", 
                      mode="max", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited

#callbacks
# callbacks_list = [checkpoint, early, reduceLROnPlat]

## Compile model

In [ ]:
model = UResNet34(input_shape=(768, 768, 3),encoder_weights=True)
model.summary()

In [ ]:
model.load_weights(PRETRAINED_SEG_MODEL_PATH)

In [ ]:
# model.save('seg_model_768_ships_ratio_0741_5hr.h5')

## Start Training

In [ ]:
# training params
input_size = 768#256
batch_size = 10#64
ships_ratio = None # TO CHANGE

lr = 1e-7# for Adam   3-4  *1e-4 for SGD (bad LRFINDER graph) [256px/64:1e-4 Adam, 384px/32:3*1e-5 Adam, 768/10:1e-5
epochs = 5
loss = bce_logdice_loss

MAX_TRAIN_STEPS = int(train_df.shape[0]/batch_size/15) # num of batches per epoch # !!!!!!!! /20 instead of /10 for LRFinder
MAX_VAL_STEPS = 100

training_steps = min(MAX_TRAIN_STEPS, train_df.shape[0]//batch_size)
validation_steps = min(MAX_VAL_STEPS, valid_df.shape[0]//batch_size)
print("training_steps:", training_steps)
print("validation_steps:", validation_steps)

In [ ]:
# yes=0
# list_l = [1 if layer.trainable==True else 0 for layer in model.layers]
# sum(list_l)/len(list_l)

for layer in model.layers:
    layer.trainable = True
    if layer.trainable is False:
        print("layer:", layer, "layer name:", layer.name)

In [ ]:
dict_lrs = {
    "ships_ratio=1": 
    {
        'epochs': 1,
        'freezed encoder': 1e-4
        
    }
    
}

In [ ]:
"""
training:
1 epoch with backbone=freezed, (lr=1e-4), ships_ratio=1 (~300 train steps here and below ) 17 mins ENOUGH (max perf:: loss: 0.3784 - dice_coef: 0.5345)
1 epoch with backbone=unfreezed, (lr=1e-4), ships_ratio=1 -> Epoch 1/1 217/217 [==============================] - 255s 1s/step - loss: 1.1537 - dice_coef: 0.2762 - binary_accuracy: 0.9551 - true_positive_rate: 0.6371 - val_loss: 0.4210 - val_dice_coef: 0.4809 - val_binary_accuracy: 0.9858 - val_true_positive_rate: 0.6577
OR 5 epochs with unfreezed, (lr=1e-4), ships_ratio=1 ,(256px, 64bs) (MAX_TRAIN_STEPS=/10) 0:53 start (~15 min epoch)
OR Epoch 4/10
347/348 [============================>.] - ETA: 2s - loss: 0.1832 - dice_coef: 0.6832 - binary_accuracy: 0.9923 - true_positive_rate: 0.8557

"""

In [ ]:
# lr_finder = LRFinder(min_lr=5*1e-5, max_lr=8*1e-3, mom=0.9)
# callbacks_list = [lr_finder]
callbacks_list = [checkpoint, early, reduceLROnPlat]


# Initial loss was bce_logdice_loss

#change input shape for data and the model
model, train_generator, validation_generator = change_input_shape(model, loss, lr=lr, new_input_size=input_size, batch_size=batch_size, ships_ratio=ships_ratio)

# fbeta_metric_callback = FBetaMetricCallback(validation_generator, int(validation_steps/10), beta=2)
# callbacks_list = [fbeta_metric_callback, checkpoint, early, reduceLROnPlat] #  calculates f_beta score just on pixels not based on IOU

#training
print("Start training...")
from datetime import datetime
print("time start model's training: ", datetime.now())
loss_history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    steps_per_epoch=training_steps,
    validation_steps=validation_steps,
    shuffle=True)

In [ ]:
model.save('seg_model_768_sr_0741_5hr.h5')


In [ ]:
# -------------------------------------------------------   SHIPS RATIO = 1
# FREEZED
# 271/348 [======================>.......] - ETA: 3:12 - 
#         loss: 0.3791 - dice_coef: 0.5334 - binary_accuracy: 0.9854 - true_positive_rate: 0.7318

# UNFREEZED
# Changing model's input shape to: 256
# Creating datagenerators...
# is_validation: False, ships_ratio:1
# is_validation: True, ships_ratio:1
# Start training...
# time start model's training:  2020-02-23 21:45:36.721266  
# Epoch 1/10
#  14/348 [>.............................] - ETA: 20:48 - loss: 0.4108 - dice_coef: 0.5399 - binary_accuracy: 0.9840 - true_positive_rate: 0.7253
# Epoch 1/10
# 348/348 [==============================] - 1029s 3s/step - loss: 0.2719 - dice_coef: 0.6001 - binary_accuracy: 0.9891 - true_positive_rate: 0.7974 - val_loss: 0.2483 - val_dice_coef: 0.6257 - val_binary_accuracy: 0.9902 - val_true_positive_rate: 0.7909

# Epoch 00001: val_dice_coef improved from 0.52284 to 0.62570, saving model to seg_model_weights.best.hdf5
# Epoch 2/10
# 348/348 [==============================] - 979s 3s/step - loss: 0.2152 - dice_coef: 0.6430 - binary_accuracy: 0.9912 - true_positive_rate: 0.8324 - val_loss: 0.1989 - val_dice_coef: 0.6793 - val_binary_accuracy: 0.9910 - val_true_positive_rate: 0.8540

# Epoch 00002: val_dice_coef improved from 0.62570 to 0.67928, saving model to seg_model_weights.best.hdf5
# Epoch 3/10
# 348/348 [==============================] - 988s 3s/step - loss: 0.1982 - dice_coef: 0.6620 - binary_accuracy: 0.9919 - true_positive_rate: 0.8455 - val_loss: 0.1788 - val_dice_coef: 0.7091 - val_binary_accuracy: 0.9916 - val_true_positive_rate: 0.8854

# Epoch 00003: val_dice_coef improved from 0.67928 to 0.70910, saving model to seg_model_weights.best.hdf5
# Epoch 4/10
# 348/348 [==============================] - 999s 3s/step - loss: 0.1833 - dice_coef: 0.6831 - binary_accuracy: 0.9923 - true_positive_rate: 0.8554 - val_loss: 0.2697 - val_dice_coef: 0.6966 - val_binary_accuracy: 0.9923 - val_true_positive_rate: 0.8651

# Epoch 00004: val_dice_coef did not improve from 0.70910
# Epoch 5/10
# 348/348 [==============================] - 1012s 3s/step - loss: 0.1759 - dice_coef: 0.6891 - binary_accuracy: 0.9928 - true_positive_rate: 0.8611 - val_loss: 0.1652 - val_dice_coef: 0.7154 - val_binary_accuracy: 0.9926 - val_true_positive_rate: 0.8684

# Epoch 00005: val_dice_coef improved from 0.70910 to 0.71536, saving model to seg_model_weights.best.hdf5
# Epoch 6/10
# 348/348 [==============================] - 1010s 3s/step - loss: 0.1684 - dice_coef: 0.6968 - binary_accuracy: 0.9928 - true_positive_rate: 0.8675 - val_loss: 0.1481 - val_dice_coef: 0.7087 - val_binary_accuracy: 0.9930 - val_true_positive_rate: 0.8580

# Epoch 00006: val_dice_coef did not improve from 0.71536

# 23:27 Keybord Interrupt

In [ ]:
# MODEL AFTER TRAINING WITH ships_ratio=1 with results described in the cell above

# ships_ratio images 0.741

# time start model's training:  2020-02-24 19:50:04.443397
# Epoch 1/20
# 723/723 [==============================] - 2070s 3s/step - loss: 0.2694 - dice_coef: 0.7855 - binary_accuracy: 0.9937 - true_positive_rate: 0.8700 - val_loss: 0.1742 - val_dice_coef: 0.8564 - val_binary_accuracy: 0.9973 - val_true_positive_rate: 0.8982
# Current F2 metric is: 0.004880824940946163

# Epoch 00001: val_dice_coef improved from -inf to 0.85642, saving model to seg_model_weights.best.hdf5
# Epoch 2/20
# 723/723 [==============================] - 1986s 3s/step - loss: 0.2576 - dice_coef: 0.7941 - binary_accuracy: 0.9940 - true_positive_rate: 0.8743 - val_loss: 0.1090 - val_dice_coef: 0.9217 - val_binary_accuracy: 0.9982 - val_true_positive_rate: 0.8664
# Current F2 metric is: 0.006318825231516655

# Epoch 00002: val_dice_coef improved from 0.85642 to 0.92169, saving model to seg_model_weights.best.hdf5
# Epoch 3/20
# 592/723 [=======================>......] - ETA: 5:36 - loss: 0.2769 - dice_coef: 0.7813 - binary_accuracy: 0.9936 - true_positive_rate: 0.8656


In [ ]:
# Changing model's input shape to: 768                       0.3630       0.3307
# Creating datagenerators...
# is_validation: False, ships_ratio:0.742
# is_validation: True, ships_ratio:0.22
# Start training...
# time start model's training:  2020-02-25 13:47:57.119213
# Epoch 1/5
#  25/604 [>.............................] - ETA: 19:58 - loss: 0.3575 - dice_coef: 0.7090 - binary_accuracy: 0.9984 - true_positive_rate: 0.8727
# 119/604 [====>.........................] - ETA: 11:17 - loss: 0.3973 - dice_coef: 0.6858 - binary_accuracy: 0.9983 - true_positive_rate: 0.8626

# Epoch 1/5
# 604/604 [==============================] - 811s 1s/step - loss: 0.3948 - dice_coef: 0.6890 - binary_accuracy: 0.9984 - true_positive_rate: 0.8335 - val_loss: 0.1545 - val_dice_coef: 0.7280 - val_binary_accuracy: 0.9995 - val_true_positive_rate: nan

# Epoch 00001: val_dice_coef improved from -inf to 0.72802, saving model to seg_model_weights.best.hdf5
# Epoch 2/5
# 604/604 [==============================] - 753s 1s/step - loss: 0.3634 - dice_coef: 0.7078 - binary_accuracy: 0.9987 - true_positive_rate: 0.8170 - val_loss: 0.3391 - val_dice_coef: 0.7608 - val_binary_accuracy: 0.9996 - val_true_positive_rate: nan

# Epoch 00002: val_dice_coef improved from 0.72802 to 0.76078, saving model to seg_model_weights.best.hdf5
# Epoch 3/5
#  59/604 [=>............................] - ETA: 9:35 - loss: 0.3333 - dice_coef: 0.7294 - binary_accuracy: 0.9988 - true_positive_rate: 0.8525

In [ ]:
loss_history.history

In [ ]:
model.summary()

In [ ]:
gc.collect()

In [ ]:
# seg_model.load_weights(weight_path)
# seg_model.save('seg_model.h5')

# Vizualization

In [ ]:
def Show_images(x,yp,yt):
    columns = 3
    rows = min(batch_size, 16)
    fig=plt.figure(figsize=(columns*4, rows*4))
    for i in range(rows):
        fig.add_subplot(rows, columns, 3*i+1)
        plt.axis('off')
        plt.imshow(x[i])
        fig.add_subplot(rows, columns, 3*i+2)
        plt.axis('off')
        plt.imshow(yp[i][:,:,0], cmap='gray')
        fig.add_subplot(rows, columns, 3*i+3)
        plt.axis('off')
        plt.imshow(yt[i][:,:,0], cmap='gray')
    plt.show()

In [ ]:
i = 5
vis_generator = train_generator 
vis_generator = validation_generator
X, y = vis_generator.__getitem__(i)
y_pred = model.predict(X)

Show_images(X, y_pred, y)

In [ ]:
i = 1
vis_generator = train_generator 
# vis_generator = validation_generator
X, y = vis_generator.__getitem__(i)
y_pred = model.predict(X)

Show_images(X, y_pred, y)